In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.widgets import RectangleSelector
from pyhdf.SD import SD, SDC
import pandas as pd
from io import BytesIO

plt.switch_backend('TkAgg')
base_dir = 'historic_data'

class RossROIManager:
    def __init__(self):
        self.rois = {
            'iaw_ross': None,
            'epw_ross': None
        }
        self.first_files_processed = {
            'iaw_ross': False,
            'epw_ross': False
        }

    def get_rois(self, category):
        return self.rois[category]

    def set_rois(self, category, rois):
        self.rois[category] = rois
        self.first_files_processed[category] = True

    def needs_roi_selection(self, category):
        return not self.first_files_processed[category]

ross_manager = RossROIManager()

def categorize_file(filename):
    lower_name = filename.lower()
    if 'iaw' in lower_name:
        if 'ccd' in lower_name:
            return 'iaw_ccd'
        elif 'ross' in lower_name:
            return 'iaw_ross'
    elif 'epw' in lower_name:
        if 'ccd' in lower_name:
            return 'epw_ccd'
        elif 'ross' in lower_name:
            return 'epw_ross'
    return 'other'

class ROISelector:
    def __init__(self, image, filename, year, category):
        self.fig, self.ax = plt.subplots(figsize=(10, 6))
        self.image = image
        self.filename = filename
        self.year = year
        self.category = category
        self.rois = []
        
        self.ax.imshow(self.image, cmap='viridis')
        self.ax.set_title(f"Select 2 ROIs for {category}\n{filename} ({year})")
        
        self.selector = RectangleSelector(
            self.ax, self.on_select,
            useblit=True, button=[1],
            minspanx=5, minspany=5,
            spancoords='pixels', interactive=True
        )
        plt.show(block=True)

    def on_select(self, eclick, erelease):
        if len(self.rois) >= 2:
            return
        
        x1 = int(min(eclick.xdata, erelease.xdata))
        x2 = int(max(eclick.xdata, erelease.xdata))
        y1 = int(min(eclick.ydata, erelease.ydata))
        y2 = int(max(eclick.ydata, erelease.ydata))
        self.rois.append((x1, y1, x2-x1, y2-y1))
        
        rect = plt.Rectangle((x1, y1), x2-x1, y2-y1,
                           linewidth=1.5, edgecolor='red', facecolor='none')
        self.ax.add_patch(rect)
        self.fig.canvas.draw()
        
        if len(self.rois) == 2:
            plt.close()

def process_hdf4(file_path):
    try:
        hdf = SD(file_path, SDC.READ)
        dataset = hdf.select('Streak_array')
        data = dataset.get()
        background = data[1, :, :]
        hdf.end()
        return background.astype(float)
    except Exception as e:
        print(f"Error processing {file_path}: {str(e)}")
        return None

def generate_histogram(pixels, filename, year):
    fig = plt.figure(figsize=(6, 4))
    plt.hist(pixels, bins=50, color='blue', alpha=0.7, edgecolor='black')
    plt.title(f"Histogram\n{filename} ({year})")
    plt.xlabel("Pixel Value")
    plt.ylabel("Frequency")
    
    buf = BytesIO()
    plt.savefig(buf, format='png', dpi=100, bbox_inches='tight')
    plt.close(fig)
    return buf.getvalue()

def analyze_file(file_path, year, category):
    background = process_hdf4(file_path)
    if background is None:
        return None, None

    filename = os.path.basename(file_path)
    histogram = None
    
    if 'ccd' in category:
        # Use full frame for CCD categories
        pixels = background.flatten()
        histogram = generate_histogram(pixels, filename, year)
        stats = {
            'mean': np.mean(pixels),
            'median': np.median(pixels),
            'stdv': np.std(pixels),
            'total': len(pixels)
        }
    else:
        # Ross categories
        if ross_manager.needs_roi_selection(category):
            selector = ROISelector(background, filename, year, category)
            ross_manager.set_rois(category, selector.rois)
        
        rois = ross_manager.get_rois(category)
        pixels = []
        for x, y, w, h in rois:
            roi_data = background[y:y+h, x:x+w]
            pixels.extend(roi_data.flatten())
        
        if pixels:
            histogram = generate_histogram(pixels, filename, year)
            stats = {
                'mean': np.mean(pixels),
                'median': np.median(pixels),
                'stdv': np.std(pixels),
                'total': len(pixels)
            }
        else:
            stats = None

    return stats, histogram

def process_year(year):
    year_dir = os.path.join(base_dir, str(year))
    if not os.path.exists(year_dir):
        return None, []
    
    hdf_files = sorted([f for f in os.listdir(year_dir) if f.endswith('.hdf')])
    year_stats = {'Year': year}
    hist_data = []
    
    category_data = {
        'iaw_ccd': [], 'iaw_ross': [],
        'epw_ccd': [], 'epw_ross': []
    }
    
    for file_name in hdf_files:
        file_path = os.path.join(year_dir, file_name)
        filename = os.path.basename(file_path)
        category = categorize_file(filename)
        
        if category not in category_data:
            continue
            
        stats, hist = analyze_file(file_path, year, category)
        if stats and hist:
            category_data[category].append(stats)
            hist_data.append({
                'Year': year,
                'Filename': filename,
                'Category': category.replace('_', ' '),
                'Histogram': hist
            })
    
    # Aggregate statistics
    for cat, entries in category_data.items():
        prefix = cat.replace('_', ' ')
        if entries:
            year_stats[f"{prefix} - mean"] = np.mean([e['mean'] for e in entries])
            year_stats[f"{prefix} - median"] = np.mean([e['median'] for e in entries])
            year_stats[f"{prefix} - stdv"] = np.mean([e['stdv'] for e in entries])
            year_stats[f"{prefix} - total"] = np.sum([e['total'] for e in entries])
        else:
            year_stats[f"{prefix} - mean"] = np.nan
            year_stats[f"{prefix} - median"] = np.nan
            year_stats[f"{prefix} - stdv"] = np.nan
            year_stats[f"{prefix} - total"] = 0
    
    return year_stats, hist_data

# Main processing
all_data = []
all_hists = []

for year in range(2015, 2026):
    print(f"Processing {year}")
    year_stat, year_hists = process_year(year)
    if year_stat:
        all_data.append(year_stat)
    if year_hists:
        all_hists.extend(year_hists)

# Create and save Excel report
if all_data:
    # Prepare statistics DataFrame
    df_stats = pd.DataFrame(all_data)
    categories = ['iaw ccd', 'iaw ross', 'epw ccd', 'epw ross']
    column_order = ['Year'] + [
        f"{cat} - {metric}" for cat in categories 
        for metric in ['mean', 'median', 'stdv', 'total']
    ]
    df_stats = df_stats.reindex(columns=column_order)
    
    # Prepare histograms DataFrame
    df_hists = pd.DataFrame(all_hists)
    
    # Create Excel writer
    with pd.ExcelWriter(
        'analysis_results.xlsx',
        engine='xlsxwriter',
        engine_kwargs={'options': {'nan_inf_to_errors': True}}
    ) as writer:
        # Write statistics data starting at row 3 (Excel row 4)
        df_stats.to_excel(writer, sheet_name='Statistics', index=False, startrow=3)
        
        workbook = writer.book
        stats_sheet = writer.sheets['Statistics']
        
        # Create header format
        header_format = workbook.add_format({
            'bold': True,
            'align': 'center',
            'valign': 'vcenter',
            'border': 1
        })
        
        # Write and merge headers
        # Year header (Excel rows 1-2)
        stats_sheet.merge_range('A1:A2', 'Year', header_format)
        
        # Category headers (Excel row 1)
        col_idx = 1
        for category in categories:
            stats_sheet.merge_range(0, col_idx, 0, col_idx+3, category, header_format)
            col_idx += 4
        
        # Metric subheaders (Excel row 2)
        col_idx = 1
        for _ in categories:
            for metric in ['mean', 'median', 'stdv', 'total']:
                stats_sheet.write(1, col_idx, metric, header_format)
                col_idx += 1
        
        # Format data cells
        num_format = workbook.add_format({'num_format': '0.000'})
        total_format = workbook.add_format({'num_format': '#,##0'})
        
        for df_row in range(len(df_stats)):
            excel_row = df_row + 3  # Data starts at Excel row 4
            # Year column
            stats_sheet.write(excel_row, 0, df_stats.iloc[df_row, 0])
            
            # Data columns
            for col in range(1, len(df_stats.columns)):
                value = df_stats.iloc[df_row, col]
                if pd.isna(value):
                    stats_sheet.write_blank(excel_row, col, None)
                elif (col % 4) == 0:  # Total columns
                    stats_sheet.write(excel_row, col, value, total_format)
                else:
                    stats_sheet.write(excel_row, col, value, num_format)
        
        # Set column widths
        stats_sheet.set_column(0, 0, 10)  # Year column
        for col in range(1, len(df_stats.columns)):
            stats_sheet.set_column(col, col, 15)

        # Histograms sheet
        df_hists.to_excel(writer, sheet_name='Histograms', index=False)
        hist_sheet = writer.sheets['Histograms']
        
        # Insert images
        for idx, row in df_hists.iterrows():
            img_data = BytesIO(row['Histogram'])
            hist_sheet.insert_image(
                idx + 1, 3,  # Start from row 1, column D
                row['Filename'],
                {'image_data': img_data, 'x_offset': 5, 'y_offset': 5}
            )
        
        # Set column widths and row heights
        hist_sheet.set_column('A:A', 10)   # Year
        hist_sheet.set_column('B:B', 30)   # Filename
        hist_sheet.set_column('C:C', 15)   # Category
        hist_sheet.set_column('D:D', 60)   # Histogram
        
        for row in range(1, len(df_hists)+1):
            hist_sheet.set_row(row, 100)

    print("Analysis results saved to analysis_results.xlsx")
else:
    print("No data processed")

Processing 2015
Processing 2016
Processing 2017
Processing 2018
Processing 2019
Processing 2020
Processing 2021
Processing 2022
Processing 2023
Processing 2024
Processing 2025
Analysis results saved to analysis_results.xlsx
